In [1]:
# python imports
import re
import json
from os.path import join
import cPickle
import numpy as np
import math
import itertools

# third party imports\
import pandas
import escher
import cloudpickle

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *
from ecolime import (ribosome, tRNA_charging, transcription, translocation, 
                     chaperones, DNA_replication, translation)
from ecolime.characterization.biomass_allocation import get_membrane_composition, get_membrane_protein

from cobrame import *
from cobrame.util import building, dogma
from cobrame.util.mass import compute_RNA_mass
from cobrame.solve.algorithms import binary_search, fva, solve_at_growth_rate
from cobrame.solve.symbolic import compile_expressions

from cloudpickle import load
import cPickle

from cobrame import ComplexData, TranscribedGene, ModificationData
from cobrame.util.building import add_modification_data

import cobra
import pandas as pd


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [7]:
control_file = './RNA_sweep_control_prototypes/proto_65_NO_tRNA_mods_NO_rRNA_mods.pickle'
with open(control_file , "rb") as infile:
    me = cPickle.load(infile)

In [8]:
me.reactions.charging_tRNA_b0202_AUC.tRNAData.modifications

{'DDusgen_path_1_at_20A': 1,
 'DDusgen_path_2_at_17': 1,
 'DDusgen_path_3_at_20': 1,
 'QtRNA_path_1_at_34': 1,
 'Y_path_2_at_65': 1,
 'Y_path_7_at_55': 1,
 'ct6A_path_1_at_37': 1,
 'm5U_path_1_at_54': 1,
 'm7G_path_1_at_46': 1}

In [3]:
sweep_df = pd.DataFrame()
#rRNA sweeps
#sweep_df.loc[ 0 , 'rRNA_mods'] = 'Negative_Sweep'
#sweep_df.loc[ 1 , 'rRNA_mods'] = 'Negative_Sweep'
#sweep_df.loc[ 2 , 'rRNA_mods'] = 'Positive_Sweep'
#sweep_df.loc[ 3 , 'rRNA_mods'] = 'Positive_Sweep'

#sweep_df.loc[ 0 , 'tRNA_mods'] = True
#sweep_df.loc[ 1 , 'tRNA_mods'] = False
#sweep_df.loc[ 2 , 'tRNA_mods'] = True
#sweep_df.loc[ 3 , 'tRNA_mods'] = False


#tRNA Sweeps
sweep_df.loc[ 4 , 'rRNA_mods'] = True
sweep_df.loc[ 5 , 'rRNA_mods'] = False
sweep_df.loc[ 6 , 'rRNA_mods'] = True
sweep_df.loc[ 7 , 'rRNA_mods'] = False
sweep_df.loc[ 4 , 'tRNA_mods'] = 'Negative_Sweep'
sweep_df.loc[ 5 , 'tRNA_mods'] = 'Negative_Sweep'
sweep_df.loc[ 6 , 'tRNA_mods'] = 'Positive_Sweep'
sweep_df.loc[ 7 , 'tRNA_mods'] = 'Positive_Sweep'

sweep_df

,rRNA_mods,tRNA_mods
4,True,Negative_Sweep
5,False,Negative_Sweep
6,True,Positive_Sweep
7,False,Positive_Sweep


In [ ]:
def build_models_for_sweep(rRNA_mod, tRNA_mod):
    path = './RNA_sweep_control_prototypes/'
    filename = 'proto_65_NO_tRNA_mods_NO_rRNA_mods.pickle'
    from tqdm import tqdm
    
    
    if tRNA_mod == "Negative_Sweep":
        for modification in tqdm(tRNA_charging.trna_modification.keys()):
            with open(path + filename , "rb") as infile:
                me = cPickle.load(infile)
                #query tRNA reactions
                for trna_rxn in me.reactions.query('charging_tRNA'):
                    if modification in trna_rxn.tRNAData.modifications.keys():
                        del trna_rxn.tRNAData.modifications[modification]
                
                #ADD RIBOSOME
                ribosome.add_ribosome(me, verbose=False, rRNA_mods = rRNA_mod)
                
                # These should be coupled to translation not ribosome formation per iOL1650
                Inf_modifications = ['Translation_initiation_factor_InfA', 'Translation_gtp_initiation_factor_InfB',
                                    'Translation_initiation_factor_InfC']
                for sub in Inf_modifications:
                    me.complex_data.ribosome.modifications[sub] = 0
                    for data in me.translation_data:
                        data.modifications[sub]= -1
                
                #Update and prune
                me.update()
                me.prune()
                
                
                #save model :
                save_path = './tRNA_modification_sweep/tRNA_negative_sweep/'
                save_file_name = 'proto65_rRNAmods_' + str(rRNA_mod) + '_tRNAnegsweep_of_' + str(modification) + '.pickle'
                with open(save_path + save_file_name, "wb") as outfile:
                    cPickle.dump(me, outfile)
                
                
        
    elif tRNA_mod == "Positive_Sweep":
        for modification in tqdm(tRNA_charging.trna_modification.keys()):
            with open(path + filename , "rb") as infile:
                me = cPickle.load(infile)
                #query tRNA reactions
                for trna_rxn in me.reactions.query('charging_tRNA'):
                    #keeps it for positive sweep
                    if modification in trna_rxn.tRNAData.modifications.keys():
                        trna_rxn.tRNAData.modifications = {modification : trna_rxn.tRNAData.modifications[modification]}
                    #deletes all others
                    else:
                        trna_rxn.tRNAData.modifications = {}
  
                    
                #ADD RIBOSOME
                ribosome.add_ribosome(me, verbose=False, rRNA_mods = rRNA_mod)
                # These should be coupled to translation not ribosome formation per iOL1650
                Inf_modifications = ['Translation_initiation_factor_InfA', 'Translation_gtp_initiation_factor_InfB',
                                    'Translation_initiation_factor_InfC']
                for sub in Inf_modifications:
                    me.complex_data.ribosome.modifications[sub] = 0
                    for data in me.translation_data:
                        data.modifications[sub]= -1
                        
                #Update and prune
                me.update()
                me.prune()
                
                
                #save model :
                save_path = './tRNA_modification_sweep/tRNA_negative_sweep/'
                save_file_name = 'proto65_rRNAmods_' + str(rRNA_mod) + '_tRNApossweep_of_' + str(modification) + '.pickle'
                with open(save_path + save_file_name, "wb") as outfile:
                    cPickle.dump(me, outfile)  
        

In [ ]:
#build models for TRNA SWEEP
for index, row in sweep_df.iterrows():
    rRNA_mod= row.rRNA_mods
    tRNA_mod= row.tRNA_mods
    build_models_for_sweep(rRNA_mod, tRNA_mod)

100%|██████████| 44/44 [1:27:26<00:00, 114.72s/it]


# tRNA charging.update()

# ribosome.add_ribosome =  formation 

#  extra ribosome modifications

# update and prune

# solve

# BEGIN CREATING PROTOTYPES FOR SWEEPS (MODIFICATION BASED)

### rRNA prototypes

In [ ]:
rrna_mod_list = ribosome.rrna_modifications.keys()

### rRNA negative sweep

##### needs tRNA modifications fully added
##### needs rRNA modifications added (except of interest)